# Gradient Explorer

In [4]:
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import ListedColormap
from scipy import linalg
from IPython.display import display, clear_output
import time
from ipywidgets import interact, fixed, FloatSlider, IntSlider

%matplotlib inline

fys=[]

def plot_function(xdims, ydims, f, title):
    global fys

    #Prepare grid for plotting decision surface
    gx1, gx2 = np.meshgrid(
        np.arange(xdims[0], xdims[1], (xdims[1] - xdims[0]) / 100.0),
        np.arange(ydims[0], ydims[1], (ydims[1] - ydims[0]) / 100.0)
    )
    gx1l = gx1.flatten()
    gx2l = gx2.flatten()
    gx = np.vstack((gx1l, gx2l)).T

    #Compute a prediction for every point in the grid
    #Cache for fast redisplay
    if (len(fys) == 0):
        y = f(gx)
        y = np.reshape(y, gx1.shape)
        fys = y
    else:
        y = fys

    #Plot a contour map of the function
    plt.contourf(gx1, gx2, y,
                 levels=np.unique(np.round(np.linspace(0, (np.max(y)), 10))))
    plt.colorbar()
    plt.contour(gx1, gx2, y, colors='k',
                levels=np.unique(np.round(np.linspace(0, (np.max(y)), 10))))

    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.grid(False)
    plt.title(title)


def plot_grad(q,x1=0,x2=0,steps=1,stepsize=1):

    fig = plt.figure(figsize=(10, 8))
    plot_function([-10, 10.1], [-10, 10.1], q.f, "Objective Function")
    
    plt.plot(x1, x2, 'wo')
    for s in range(steps):
        d = -q.g(np.array([x1,x2]))
        if(np.abs(x1-d[0])>0.5 or np.abs(x2-d[1])>0.5):
            plt.arrow( x1, x2, stepsize*d[0], stepsize*d[1], color='w', head_length=0.5, head_width=0.5,length_includes_head=True)
        x1 = x1 + stepsize*d[0]
        x2 = x2 + stepsize*d[1]
    
    plt.xlim(-10,10)
    plt.ylim(-10,10)
    plt.show()

Create a Function
===

In [5]:
class quad_func():
    def __init__(self, A, b, c):
        self.A = A
        self.b = b
        self.c = c
        self.fcount = 0
        self.gcount = 0

    def f(self, x):
        self.fcount += 1
        if (len(x.shape) == 1):
            fx = x.T.dot(self.A).dot(x) + x.dot(self.b.T) + self.c
            return fx[0]
        else:
            return np.sum(x.dot(self.A) * x, axis=1, keepdims=True) + x.dot(self.b.T) + self.c

    def g(self, x):
        self.gcount += 1
        return 2 * np.reshape(x.dot(self.A) + self.b, x.shape)

    def reset_counts(self):
        self.fcount = 0
        self.gcount = 0


Gradient Explorer
--

In [3]:
q = quad_func(np.array([[0.1, 0.05], [0.05, 0.1]]), np.array([[0, 0]]), 7)

x1s=FloatSlider(min=-10, max=10, step=0.5, continuous_update=False)
x2s=FloatSlider(min=-10, max=10, step=0.5, continuous_update=False)
ss =IntSlider(min=1, max=20, continuous_update=False)

interact(plot_grad, q=fixed(q), x1=x1s,x2=x2s,steps=ss,stepsize=[0.1,1,2.5,5,10]);

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='x1', max=10.0, min=-10.0, s…